In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.cross_validation import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import sklearn.metrics as sklm

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn import cross_validation
import sklearn.model_selection as ms

In [ ]:
loan_data_feature=pd.read_csv('loan_data_feature.csv')
X = np.array(loan_data_feature.drop(columns = ['good_credit','long_term','pay_month_01','pay_month_02','pay_month_03',
                                               'pay_month_04','pay_month_05','pay_month_06','bill_amount_01','bill_amount_02',
                                               'bill_amount_03','bill_amount_04','bill_amount_05','bill_amount_06']))
y = np.array(loan_data_feature['good_credit'].astype(np.int32)).reshape((-1, ))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
def print_metrics(y_true, y_pred):
    metrics = sklm.precision_recall_fscore_support(y_true, y_pred)
    conf = sklm.confusion_matrix(y_true, y_pred)
    print('                 Confusion matrix')
    print('                 Score positive    Score negative')
    print('Actual positive    %6d' % conf[0,0] + '             %5d' % conf[0,1])
    print('Actual negative    %6d' % conf[1,0] + '             %5d' % conf[1,1])
    print('')
    print('Accuracy         %0.2f' % sklm.accuracy_score(y_true, y_pred))
    print('precision        %0.2f' % sklm.precision_score(y_true, y_pred))
    print('recall           %0.2f' % sklm.recall_score(y_true, y_pred))
    print('f1               %0.2f' % sklm.f1_score(y_true, y_pred))
    print('AUC              %0.2f' % sklm.roc_auc_score(y_true, y_pred))
    print('Macro precision  %0.2f' % float((float(metrics[0][0]) + float(metrics[0][1]))/2.0))
    print('Macro recall     %0.2f' % float((float(metrics[1][0]) + float(metrics[1][1]))/2.0))
    print(' ')
    print('           Positive      Negative')
    print('Num case   %6d' % metrics[3][0] + '        %6d' % metrics[3][1])
    print('Precision  %6.2f' % metrics[0][0] + '        %6.2f' % metrics[0][1])
    print('Recall     %6.2f' % metrics[1][0] + '        %6.2f' % metrics[1][1])
    print('F1         %6.2f' % metrics[2][0] + '        %6.2f' % metrics[2][1])

def plot_auc(y_true, y_pred):
    ## Compute the false positive rate, true positive rate
    ## and threshold along with the AUC
    fpr, tpr, threshold = sklm.roc_curve(y_true, y_pred)
    auc = sklm.auc(fpr, tpr)
    
    ## Plot the result
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, color = 'orange', label = 'AUC = %0.2f' % auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
def print_format(f,x,y,z):
    print('Fold %2d    %4.3f        %4.3f      %4.3f' % (f, x, y, z))

def print_cv(scores):
    fold = [x + 1 for x in range(len(scores['test_precision_macro']))]
    print('         Precision     Recall       AUC')
    [print_format(f,x,y,z) for f,x,y,z in zip(fold, scores['test_precision_macro'], 
                                          scores['test_recall_macro'],
                                          scores['test_roc_auc'])]
    print('-' * 40)
    print('Mean       %4.3f        %4.3f      %4.3f' % 
          (np.mean(scores['test_precision_macro']), np.mean(scores['test_recall_macro']), np.mean(scores['test_roc_auc'])))  
    print('Std        %4.3f        %4.3f      %4.3f' % 
          (np.std(scores['test_precision_macro']), np.std(scores['test_recall_macro']), np.std(scores['test_roc_auc'])))

### Logistic Regression

In [ ]:
model = LogisticRegression()
param_grid = {'C': [0.01, 0.1, 1, 10, 100,1000], 'penalty': ['l1', 'l2']}
gridsearch = GridSearchCV(model, param_grid)
gridsearch.fit(X_train, y_train);
print("Best Hyper Parameters:\n",gridsearch.best_params_)

In [ ]:
model = LogisticRegression(C=1,penalty='l1')
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print_metrics(y_test, y_pred)
plot_auc(y_test, y_pred)

In [ ]:
scoring = ['precision_macro', 'recall_macro', 'roc_auc']
scores = ms.cross_validate(model, X_train, y_train, scoring=scoring,cv=5, return_train_score=False)
print_cv(scores)

### K-Nearest Neighbors

In [ ]:
model = KNeighborsClassifier(n_jobs=-1)
param_grid = {'n_neighbors':[5,6,7,8,9,10],
          'leaf_size':[1,2,3,5],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute'],
          'n_jobs':[-1]}
gridsearch = GridSearchCV(model, param_grid=param_grid, n_jobs=1)
#Learning
gridsearch.fit(X_train,y_train)
#The best hyper parameters set
print("Best Hyper Parameters:\n",gridsearch.best_params_)

In [ ]:
model = KNeighborsClassifier(algorithm='auto',
                            leaf_size=1,
                            n_jobs=-1,
                            n_neighbors=10,
                            weights='distance')
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)
print_metrics(y_test, y_pred)
plot_auc(y_test, y_pred)

In [ ]:
scoring = ['precision_macro', 'recall_macro', 'roc_auc']
scores = ms.cross_validate(model, X_train, y_train, scoring=scoring,cv=5, return_train_score=False)
print_cv(scores)

### Decision Tree

In [ ]:
model = DecisionTreeClassifier()
param_grid = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],
          'random_state':[123]} 
gridsearch = GridSearchCV(model, param_grid=param_grid, n_jobs=-1)
#Learning
gridsearch.fit(X_train,y_train)
#The best hyper parameters set
print("Best Hyper Parameters:\n",gridsearch.best_params_)

In [ ]:
model = DecisionTreeClassifier(max_features='log2',
                            min_samples_leaf=8,
                            min_samples_split=2,
                            random_state=123)
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)
print_metrics(y_test, y_pred)
plot_auc(y_test, y_pred)

### Random Forest Classifier

In [ ]:
model = RandomForestClassifier()
param_grid = {'criterion': ['gini', 'entropy'],
              'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
gridsearch = GridSearchCV(model, param_grid=param_grid, n_jobs=-1)
#Learning
gridsearch.fit(X_train,y_train) 
#The best hyper parameters set
print("Best Hyper Parameters:\n",gridsearch.best_params_)

In [ ]:
model =  RandomForestClassifier(bootstrap= False,
                                max_depth= None,
                                max_features='sqrt',
                                min_samples_leaf= 1,
                                min_samples_split= 2,
                                n_estimators= 2000)
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)
print_metrics(y_test, y_pred)
plot_auc(y_test, y_pred)

### Support Vector Machine

In [ ]:
model = SVC()
param_grid = {'C': [6,7,8,9,10,11,12], 
            'kernel': ['linear','rbf']}
gridsearch = GridSearchCV(model, param_grid=param_grid)
#Learning
gridsearch.fit(X_train,y_train)
#The best hyper parameters set
print("Best Hyper Parameters:\n",gridsearch.best_params_)

In [ ]:
model = SVC(C=6, kernel='rbf')
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)
print_metrics(y_test, y_pred)
plot_auc(y_test, y_pred)

### Linear Discriminant Analysis